In [ ]:
from sklearn.metrics import plot_roc_curve

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option("display.max_rows", None, "display.max_columns", None) 

fighter_details = pd.read_csv("../input/ufcdata/raw_fighter_details.csv")
df = pd.read_csv("../input/ufcdata/raw_total_fight_data.csv", sep=';')

First lets see the dataframe of the UFC Fight from 1993-2019

In [ ]:
df.head() # The Match Data

In [ ]:
fighter_details.head() #The fighter data 

Come take a look the info of the match data

In [ ]:
df.info()

# Column definitions:

- `R_` and `B_` prefix signifies red and blue corner fighter stats respectively
- `KD` is number of knockdowns
- `SIG_STR` is no. of significant strikes 'landed of attempted'
- `SIG_STR_pct` is significant strikes percentage
- `TOTAL_STR` is total strikes 'landed of attempted'
- `TD` is no. of takedowns
- `TD_pct` is takedown percentages
- `SUB_ATT` is no. of submission attempts
- `PASS` is no. times the guard was passed?
- `REV?`
- `HEAD` is no. of significant strinks to the head 'landed of attempted'
- `BODY` is no. of significant strikes to the body 'landed of attempted'
- `CLINCH` is no. of significant strikes in the clinch 'landed of attempted'
- `GROUND` is no. of significant strikes on the ground 'landed of attempted'
- `win_by` is method of win
- `last_round` is last round of the fight (ex. if it was a KO in 1st, then this will be 1)
- `last_round_time` is when the fight ended in the last round
- `Format` is the format of the fight (3 rounds, 5 rounds etc.)
- `Referee` is the name of the Ref
- `date` is the date of the fight
- `location` is the location in which the event took place
- `Fight_type` is which weight class and whether it's a title bout or not
- `Winner` is the winner of the fight

# First lets gather the data from the raw match data above 
1. Split the attemp and strike landed
2. Make Percentage to Fraction
3. Get the Fighter Division
4. Get Number of Round
5. Replace The Winner Name Column with R/B
****

# Data cleaning 
1. Split strike attempted and landed because it is given in format "n of m"
2. Make percentages to fractions

## 1. Split The Strike Attemp and Strike Landed

In [ ]:
df.columns

In [ ]:
columns = ['R_SIG_STR.', 'B_SIG_STR.', 'R_TOTAL_STR.', 'B_TOTAL_STR.',
       'R_TD', 'B_TD', 'R_HEAD', 'B_HEAD', 'R_BODY','B_BODY', 'R_LEG', 'B_LEG', 
        'R_DISTANCE', 'B_DISTANCE', 'R_CLINCH','B_CLINCH', 'R_GROUND', 'B_GROUND']

In [ ]:
attemp = '_att'
landed = '_landed'

for column in columns:
    df[column+attemp] = df[column].apply(lambda X: int(X.split('of')[1]))
    df[column+landed] = df[column].apply(lambda X: int(X.split('of')[0]))
    
df.drop(columns, axis=1, inplace=True)

# Make The Percentage to Fraction

In [ ]:
pct_columns = ['R_SIG_STR_pct','B_SIG_STR_pct', 'R_TD_pct', 'B_TD_pct']

for column in pct_columns:
    df[column] = df[column].apply(lambda X: float(X.replace('%', ''))/100)

# Get Fighter Division

In [ ]:
def Division(X):
    for Division in weight_classes:
        if Division in X:
            return Division
    if X == 'Catch Weight Bout' or 'Catchweight Bout':
        return 'Catch Weight'
    else:
        return 'Open Weight'

In [ ]:
weight_classes = ['Women\'s Strawweight', 'Women\'s Bantamweight', 
                  'Women\'s Featherweight', 'Women\'s Flyweight', 'Lightweight', 
                  'Welterweight', 'Middleweight','Light Heavyweight', 
                  'Heavyweight', 'Featherweight','Bantamweight', 'Flyweight', 'Open Weight']

df['weight_class'] = df['Fight_type'].apply(Division)

In [ ]:
df['weight_class'].value_counts()

# Number of Round

In [ ]:
def get_rounds(X):
    if X == 'No Time Limit':
        return 1
    else:
        return len(X.split('(')[1].replace(')', '').split('-'))

df['no_of_rounds'] = df['Format'].apply(get_rounds)

# Replace Winner Name with R/B

In [ ]:
df['Winner'].isnull().sum()

In [ ]:
df['Winner'].fillna('Draw', inplace=True) #fill the null value with draw

In [ ]:
def get_renamed_winner(row):
    if row['R_fighter'] == row['Winner']:
        return 'Red'
    elif row['B_fighter'] == row['Winner']:
        return 'Blue'
    elif row['Winner'] == 'Draw':
        return 'Draw'

df['Winner'] = df[['R_fighter', 'B_fighter', 'Winner']].apply(get_renamed_winner, axis=1)

In [ ]:
df['Winner'].value_counts()

# Deal with fighter details data
1. `Inch` to `CM`
2. Merger `Fighter Data` with `Match Data`
3. Get Fighter `Age`

# Inch to CM

In [ ]:
def convert_to_cms(X):
    if X is np.NaN:
        return X
    elif len(X.split("'")) == 2:
        feet = float(X.split("'")[0])
        inches = int(X.split("'")[1].replace(' ', '').replace('"',''))
        return (feet * 30.48) + (inches * 2.54)
    else:
        return float(X.replace('"','')) * 2.54

In [ ]:
fighter_details['Height'] = fighter_details['Height'].apply(convert_to_cms)
fighter_details['Reach'] = fighter_details['Reach'].apply(convert_to_cms)

In [ ]:
fighter_details['Weight'] = fighter_details['Weight'].apply(lambda X: float(X.replace(' lbs.', '')) if X is not np.NaN else X)

In [ ]:
fighter_details.head()

# Merger Fighter Data with Match Data

In [ ]:
new = df.merge(fighter_details, left_on='R_fighter', right_on='fighter_name', how='left')

In [ ]:
new = new.drop('fighter_name', axis=1)

In [ ]:
new.rename(columns={'Height':'R_Height',
                          'Weight':'R_Weight',
                          'Reach':'R_Reach',
                          'Stance':'R_Stance',
                          'DOB':'R_DOB'}, 
                 inplace=True)

In [ ]:
new = new.merge(fighter_details, left_on='B_fighter', right_on='fighter_name', how='left')

In [ ]:
new = new.drop('fighter_name', axis=1)

In [ ]:
new.rename(columns={'Height':'B_Height',
                          'Weight':'B_Weight',
                          'Reach':'B_Reach',
                          'Stance':'B_Stance',
                          'DOB':'B_DOB'}, 
                 inplace=True)

In [ ]:
new.head()

# Get Age

In [ ]:
new['R_DOB'] = pd.to_datetime(new['R_DOB'])
new['B_DOB'] = pd.to_datetime(new['B_DOB'])
new['date'] = pd.to_datetime(new['date'])

In [ ]:
new['R_year'] = new['R_DOB'].apply(lambda x: x.year)
new['B_year'] = new['B_DOB'].apply(lambda x: x.year)
new['date_year'] = new['date'].apply(lambda x: x.year)

In [ ]:
def get_age(row):
    B_age = (row['date_year'] - row['B_year'])
    R_age = (row['date_year'] - row['R_year'])
    if np.isnan(B_age)!=True:
        B_age = B_age
    if np.isnan(R_age)!=True:
        R_age = R_age
    return pd.Series([B_age, R_age], index=['B_age', 'R_age'])

In [ ]:
new[['B_age', 'R_age']]= new[['date_year', 'R_year', 'B_year']].apply(get_age, axis=1)

In [ ]:
new.drop(['R_DOB', 'B_DOB','date_year','R_year','B_year'], axis=1, inplace=True)

# Fighter Country

In [ ]:
new['country'] = new['location'].apply(lambda x : x.split(',')[-1])

# UFC Prediction

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, RandomizedSearchCV
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings("ignore")

# Deal with null Values

In [ ]:
df = new.copy();

In [ ]:
df.isnull().sum();

PREVIOUSLY SPLIT WAS MADE AFTER LABEL ENCODING AND USING MEANS TO IMPUTE MISSING VALUES THIS IS NOT GOOD PRACTICE BECAUSE THE MEAN IS CALCULATED USING TEST EXAMPLES AND LABELS MAY NOT EXIST IN THE TEST EXAMPLE, DATA LEACKAGE. I MOVED THE SPLITTING HERE. FURTHERMORE, SPLIT SHOULD BE DONE CHRONOLOGICALLY AND NOT RANDOMLY. 


In [ ]:
#Encode targets
df['Winner_num'] = df.Winner.map({'Red':0,'Blue':1,'Draw':2})
df=df.drop(columns=['Winner'],axis=1)

In [ ]:
df['Winner_num'];

In [ ]:
#First let's sort by date:
df=df.sort_values(by=['date'], axis=0)


X = df.drop(['Winner_num'], axis=1)
y = df['Winner_num']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.25, shuffle=False)

In [ ]:
#Let's impute mode or mean as done before but this time after splittng

#Mode for Stance
from sklearn.impute import SimpleImputer
mode_imp=SimpleImputer(strategy='most_frequent')
mode_columns=['B_Stance','R_Stance']
X_train[mode_columns]=mode_imp.fit_transform(X_train[mode_columns])
X_test[mode_columns]=mode_imp.transform(X_test[mode_columns])




#ORIGINAL---------------------------------------------------------------------
#from statistics import mode 
#X_train['B_Stance'] = X_train['B_Stance'].fillna(df['B_Stance'].mode()[0])
#X_test['R_Stance'] = X_test['R_Stance'].fillna(df['R_Stance'].mode()[0])


In [ ]:
#Mean for every other numerical category
mean_cols=[colname for colname in X_train.columns if X_train[colname].dtype in ['int64','float64']]


mean_imp=SimpleImputer(strategy='mean')
X_train[mean_cols]=mean_imp.fit_transform(X_train[mean_cols])
X_test[mean_cols]=mean_imp.transform(X_test[mean_cols])




# Data Encoding

The main issue with encoding is that training set must have the same categories in all categorical values as the test set. Otherwise, label encoder fails to assign a label to categories that were not seen during training. To make things easier, let's drop categories with different values for train and test. 

In [ ]:
X_train.columns

In [ ]:
print(set(X_train['R_Stance']))
print(set(X_test['R_Stance']))
set(X_train['R_Stance'])!=set(X_test['R_Stance'])

In [ ]:
#find categorical columns
cat_cols=[col for col in X_train.columns if X_train[col].dtype=='object']

different_columns=[]
#Let's make a list of columns whose contained values differ from training to testing set
for col in cat_cols:
    if set(X_train[col])!= set(X_test[col]):
        different_columns.append(col)
print(different_columns)
print(cat_cols)
#Drop different columns
X_train=X_train.drop(columns=different_columns, axis=1)
X_test=X_test.drop(columns=different_columns,axis=1)

In [ ]:
#We would need a handcrafted encoder to encode the columns that are different between training and validation set. 

#data_enc1 = df['weight_class']
#data_enc1 = enc.fit_transform(data_enc1)
#
#data_enc2 = df['R_Stance']
#data_enc2 = enc.fit_transform(data_enc2)
#
#data_enc3 = df['B_Stance']
#data_enc3= enc.fit_transform(data_enc3)

In [ ]:
#data_enc1 = pd.DataFrame(data_enc1, columns=['weight_class'])
#data_enc2 = pd.DataFrame(data_enc2, columns=['R_Stance'])
#data_enc3 = pd.DataFrame(data_enc3, columns=['B_Stance'])

In [ ]:
#df[['weight_class']] = data_enc1[['weight_class']]
#df[['R_Stance']] = data_enc2[['R_Stance']]
#df[['B_Stance']] = data_enc3[['B_Stance']]

In [ ]:
#df = pd.concat([df,pd.get_dummies(df['win_by'], prefix='win_by')],axis=1)
#df.drop(['win_by'],axis=1, inplace=True)

In [ ]:
#encode = df[['R_fighter','B_fighter','weight_class']].apply(enc.fit_transform)
#encode.head()

In [ ]:
#df[['R_fighter','B_fighter','weight_class']] = encode[['R_fighter','B_fighter','weight_class']] 

In [ ]:
plt.figure(figsize=(10,15))
sns.heatmap(df.corr()[['Winner_num']].sort_values(by='Winner_num', ascending=False),annot=True)
plt.show()

# Normalize data with Standard Scaler

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler();
numerical_cols=[colname for colname in X_train.columns if X_train[colname].dtype in ['int64','float64']];
X_train[numerical_cols]=scaler.fit_transform(X_train[numerical_cols]);
X_test[numerical_cols]=scaler.transform(X_test[numerical_cols]);

#numerical = df.drop(['R_fighter','B_fighter','weight_class','no_of_rounds','Winner_num'], axis=1)

## Drop Unecessary Column
We already did that before, use feature engineering to choose best categories

In [ ]:
X_train=X_train.drop(columns='date');
X_test=X_test.drop(columns='date');

# Modeling XGBoost

In [ ]:
model = XGBClassifier()
X_train=X_train.loc[y_train!=2,:]
y_train=y_train[y_train!=2]
X_test=X_test.loc[y_test!=2]
y_test=y_test[y_test!=2]

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
#Already did this before


#X = df_fix.drop(['Winner_num'], axis=1)
#y = df_fix['Winner_num']

#X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.25, random_state = 42)

In [ ]:
model.fit(X_train,y_train)

In [ ]:
pred = model.predict(X_test)

# Evaluate model

In [ ]:
Score = model.score(X_test,y_test)
print("Score: %.2f%%" % (Score * 100.0))

In [ ]:
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import plot_confusion_matrix
plot_roc_curve(model,X_test,y_test)
plot_confusion_matrix(model,X_test,y_test)